In [1]:
import sqlite3
from pathlib import Path
import pandas as pd

/Users/santiagobasulto/Library/Caches/pypoetry/virtualenvs/pycon-concurrency-tutorial-2020-a5tTVfGc-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
SCHEMA = '''

drop table if exists price;
create table price (
  id integer primary key autoincrement,
  exchange text not null,
  symbol text not null,
  open DECIMAL(10, 4),
  high DECIMAL(10, 4),
  low DECIMAL(10, 4),
  close DECIMAL(10, 4),
  volume DECIMAL(10, 4),
  day DATE not null
);
'''

### Prune DB

In [3]:
conn = sqlite3.connect('prices.db')

In [4]:
conn.executescript(SCHEMA)

In [5]:
conn.commit()

In [6]:
conn.close()

### Insert Data

In [7]:
BASE_PATH = Path('crypto_data/')

In [8]:
files = list(BASE_PATH.glob('*.csv'))

In [9]:
INSERT_STATEMENT = """
INSERT INTO price (
    exchange, symbol, open, high, low, close, volume, day
) VALUES (?, ?, ?, ?, ?, ?, ?, ?);
"""

In [10]:
conn = sqlite3.connect('prices.db')

In [11]:
for file in files:
    exchange, symbol = file.name[:-4].split('_')
    df = pd.read_csv(str(file))
    df['exchange']  = exchange
    df['symbol']  = symbol
    
    values = df[['exchange', 'symbol', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'DateTime']].values
    conn.executemany(INSERT_STATEMENT, values)
    conn.commit()

In [12]:
conn.close()

### Final Test

In [13]:
conn = sqlite3.connect('prices.db')

In [14]:
cursor = conn.cursor()

In [15]:
cursor.execute('SELECT COUNT(*) FROM price;')

In [16]:
cursor.fetchone()

(28008,)

In [17]:
cursor.execute('SELECT * FROM price LIMIT 5;')

In [18]:
cursor.fetchall()

[(1,
  'hitbtc',
  'ltc',
  167.625,
  173.372,
  167.09099999999998,
  173.313,
  1893.8,
  '2018-03-24'),
 (2,
  'hitbtc',
  'ltc',
  173.264,
  173.77900000000002,
  162.202,
  162.668,
  14864.014,
  '2018-03-25'),
 (3,
  'hitbtc',
  'ltc',
  162.60399999999998,
  167.28,
  160.417,
  163.946,
  14783.375,
  '2018-03-26'),
 (4,
  'hitbtc',
  'ltc',
  164.107,
  164.69400000000002,
  145.002,
  151.275,
  17593.473,
  '2018-03-27'),
 (5,
  'hitbtc',
  'ltc',
  151.224,
  152.44299999999998,
  137.503,
  137.561,
  20596.676,
  '2018-03-28')]

In [19]:
conn.close()

### Exchanges

In [20]:
conn = sqlite3.connect('prices.db')

In [21]:
cursor = conn.cursor()

In [22]:
cursor.execute('SELECT DISTINCT exchange FROM price;')

In [23]:
cursor.fetchall()

[('hitbtc',),
 ('mexbt',),
 ('kraken',),
 ('okex',),
 ('bittrex',),
 ('bitstamp',),
 ('bitfinex',),
 ('coinbase-pro',),
 ('poloniex',),
 ('cexio',),
 ('huobi',)]

In [24]:
cursor.execute('SELECT DISTINCT symbol FROM price;')

In [25]:
cursor.fetchall()

[('ltc',), ('btc',), ('eth',)]

### Filtered query:

In [26]:
cursor.execute('SELECT * FROM price WHERE symbol = "btc" AND exchange = "bitfinex" AND day = "2019-07-20";')

In [27]:
cursor.fetchall()

[(6064,
  'bitfinex',
  'btc',
  10661,
  10751,
  10115,
  10516,
  387.16246975,
  '2019-07-20')]